In [1]:
import json
with open(".\\data\\file_references.json") as fd:
    repos = json.load(fd)

def get_element(tag):
    elements = []
    for parent in tag.parents:
        if parent.name is None:
            break
        siblings = [sib for sib in parent.children if sib.name == tag.name]
        if len(siblings) > 1:
            elements.append(f'{tag.name}[{siblings.index(tag) + 1}]')
        else:
            elements.append(tag.name)
        tag = parent
    elements.reverse()
    return '/' + '/'.join(elements)



In [2]:
import requests
from bs4 import BeautifulSoup
import json

with open("./data/file_references.json", "r") as fd:
    repos = json.load(fd)

for idx in range(181,182):
    url = repos[idx]
    style_assoc = {}
    response = requests.get(url)
        # Check if the request was successful
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')
        tags_with_specific_property = soup.find_all(attrs={"style": True})
        if tags_with_specific_property:
            print(f"\n[{idx}] : {url}")
        for tag in tags_with_specific_property:
            xpath = get_element(tag)
            print(f"{xpath} : {tag['style'].strip()}")
            for s in tag['style'].strip().split(";"):
                style_assoc.setdefault(get_element(tag),[]).append(s)
        idx = idx + 1
    else:
        print(f"Failed to fetch the URL. Status code: {response.status_code}")





[181] : https://raw.githubusercontent.com/swimlane/ngx-charts/ff89cfe08ada08ab898083be76cd0a6cc1adfdb2/src/app/app.component.html
/main/div[1]/div : position: absolute; top: 50px; left: 50px; right: 50px; bottom: 50px
/main/div[1]/div/div[2]/ngx-charts-line-chart : position: static; display: inline-block; transform: none
/main/div[1]/div/div[2]/ngx-charts-polar-chart : position: static; display: inline-block; transform: none
/main/div[1]/div/div[3] : padding: 10px; border: 1px solid white
/main/div[1]/div/div[3]/h4 : text-align: center
/main/div[1]/div/div[3]/div : height: 17px; font-size: 12px
/main/div[1]/div/div[4] : padding: 10px; border: 1px solid white
/main/div[1]/div/div[4]/div/button : float: right
/main/div[2]/div : margin: 20px
/main/div[2]/div/h3[3] : cursor: pointer
/main/div[2]/div/div[1] : margin-left: 10px
/main/div[2]/div/div[2]/h3 : cursor: pointer
/main/div[2]/div/div[2]/div : margin-left: 10px
/main/div[2]/div/h3[4] : cursor: pointer
/main/div[2]/div/select[3] : ma

Verify that the set is correct

In [3]:
import collections
od = collections.OrderedDict(sorted(style_assoc.items()))
for k , v in od.items():
    print(f"{k} : {v}")





/main/div[1]/div : ['position: absolute', ' top: 50px', ' left: 50px', ' right: 50px', ' bottom: 50px']
/main/div[1]/div/div[2]/ngx-charts-line-chart : ['position: static', ' display: inline-block', ' transform: none']
/main/div[1]/div/div[2]/ngx-charts-polar-chart : ['position: static', ' display: inline-block', ' transform: none']
/main/div[1]/div/div[3] : ['padding: 10px', ' border: 1px solid white']
/main/div[1]/div/div[3]/div : ['height: 17px', ' font-size: 12px']
/main/div[1]/div/div[3]/h4 : ['text-align: center']
/main/div[1]/div/div[4] : ['padding: 10px', ' border: 1px solid white']
/main/div[1]/div/div[4]/div/button : ['float: right']
/main/div[2]/div : ['margin: 20px']
/main/div[2]/div/div[1] : ['margin-left: 10px']
/main/div[2]/div/div[2]/div : ['margin-left: 10px']
/main/div[2]/div/div[2]/h3 : ['cursor: pointer']
/main/div[2]/div/div[3] : ['margin-left: 10px']
/main/div[2]/div/div[4] : ['margin-left: 10px']
/main/div[2]/div/div[4]/div[18]/select : ['margin-left: 10px']
/mai

In [26]:
# Invert the dictionary.
from collections import defaultdict

inverted_dictionary = defaultdict(list)

for tag_position , styles in od.items():
    for style in styles:
        inverted_dictionary[style.strip()].append(tag_position)

sorted_inverted_dict = dict(sorted(inverted_dictionary.items(), key=lambda item: len(item[1]), reverse=False))

grouped_1_len = defaultdict(list)

for k , v in sorted_inverted_dict.items():
    if len(v) <= 1:
        print(f"{k} : {v}")

def generate_css_classes(mapping):
    # Group elements by their CSS properties
    grouped_properties = defaultdict(list)
    for css_property, elements in mapping.items():
        for element in elements:
            grouped_properties[element].append(css_property)
    
    # Generate CSS classes
    css_classes = {}
    for i, (element, properties) in enumerate(grouped_properties.items()):
        class_name = f'class-{i+1}'
        css_classes[class_name] = properties

    return css_classes

def merge_identical_classes(class_mappings):
    property_to_class = defaultdict(list)

    # Map property sets to class names
    for class_name, properties in class_mappings.items():
        property_key = tuple(sorted(properties))
        property_to_class[property_key].append(class_name)

    # Create a new dictionary to store the merged classes
    merged_classes = {}
    for properties, class_names in property_to_class.items():
        if len(class_names) > 1:
            new_class_name = class_names[0]
            merged_classes[new_class_name] = list(properties)
        else:
            merged_classes[class_names[0]] = list(properties)

    return merged_classes

css_classes = generate_css_classes(sorted_inverted_dict)
css_classes = merge_identical_classes(css_classes)
for k , v in css_classes.items():
    print(f"{k} : {v}")


position: absolute : ['/main/div[1]/div']
top: 50px : ['/main/div[1]/div']
left: 50px : ['/main/div[1]/div']
right: 50px : ['/main/div[1]/div']
bottom: 50px : ['/main/div[1]/div']
height: 17px : ['/main/div[1]/div/div[3]/div']
font-size: 12px : ['/main/div[1]/div/div[3]/div']
text-align: center : ['/main/div[1]/div/div[3]/h4']
float: right : ['/main/div[1]/div/div[4]/div/button']
margin: 20px : ['/main/div[2]/div']
class-1 : ['bottom: 50px', 'left: 50px', 'position: absolute', 'right: 50px', 'top: 50px']
class-2 : ['font-size: 12px', 'height: 17px']
class-3 : ['text-align: center']
class-4 : ['float: right']
class-5 : ['margin: 20px']
class-6 : ['display: inline-block', 'position: static', 'transform: none']
class-8 : ['border: 1px solid white', 'padding: 10px']
class-10 : ['cursor: pointer']
class-14 : ['margin-left: 10px']
